__Автор__: Карпаев Алексей, ассистент кафедры информатики и вычислительной математики

# Численное интегрирование: программная реализация, ОО подход

## Постановка задачи 

Приближенно вычислить первообразную функции $f$ согласно формуле:

$$
F(x) = \int_{a}^{x} f(t) dt
$$


## Метод численного интегрирования
Для приближенного вычисления интеграла будем пользоваться квадратурными формулами. На отрезке $[a, x]$ введем равномерную сетку с шагом $h = \frac{x-a}{N}$, где $N$ --- число отрезков разбиения. Используя значения функции $f(t)$ в узлах сетки вычислим приближенное значение определенного интеграла на отрезке $[a, x]$ с помощью формулы трапеций. Простая неоптимизированная реализация вычислений по данной формуле с использованием процедурного подхода:

In [ ]:
def Integrate(a, x, f, numBlocks):
    # N = numBlocks
    gridArray = np.linspace(a, x, numBlocks + 1)
    h = (float(x) - a)/numBlocks
    
    value = 0.
    for i in range(len(gridArray) - 1):
        tCurrent = gridArray[i]
        tNext = gridArray[i + 1]
        value += 0.5*(f(tCurrent) + f(tNext))*h
    
    return value

Использование:

In [ ]:
import numpy as np

xL, xR = 1., 2.
numBlocks = 100
listOfFunctions = [lambda t: np.cos(np.sin(t**2)), lambda t: np.sin(np.cos(t**2)), \
                   lambda t: 2./(3. + np.cos(np.sin(t**2))), \
                  lambda t: 2./(1. - np.exp(np.sin(t**2)))]


for function in listOfFunctions:
    print (Integrate(xL, xR, function, numBlocks))

Теперь рассмотрим реализацию класса для представления первообразной. Функция $F(x)$ является функцией одной переменной $x$ с параметрами $a$ и $f$:

$$
F(x) = F(x; a, f) = \int_{a}^{x} f(t)dt
$$

Как и в предыдущих лекциях, хранить значения параметров будем в полях класса, а вычислять значение функции в точке $x$ будем с помощью отдельного метода. Итак, класс __Integral__:

__Поля класса__:
* интегрируемая функция $f$ (параметр)
* левая граница интегрирования $a$ (параметр)
* число отрезков разбиения $N (numBlocks)$(дополнительный параметр)
* ... (по Вашему усмотрению)

__Методы класса__:
* "сеттеры" для интегрируемой функции, левой границы интегрирования, числа отрезков разбиения
* вычислить значение первообразной в точке $x$ (используя метод __--call--__)
* ... (по Вашему усмотрению)

In [ ]:
import time

class Integral:
    
    def __init__(self):
        print('A blanc object of class ' + self.__class__.__name__ + ' is created.')
        
    def SetFunction(self, f):
        self._f = f
    
    def SetLeftBorder(self, a):
        self._a = float(a)
    
    def SetNumBlocks(self, numBlocks):
        self._numBlocks = numBlocks
          
    def __call__(self, x):
        x = float(x)
        gridArray = np.linspace(self._a, x, self._numBlocks + 1)
        gridLength = len(gridArray)
        h = (x - self._a)/self._numBlocks
        
        # вычисление по формуле трапеций
        print('Calculations took...', end = '')
        start = time.clock()
        
        value = 0.
        for i in range(gridLength - 1):
            tCurrent = gridArray[i]
            tNext = gridArray[i + 1]
            value += 0.5*(self._f(tCurrent) + self._f(tNext))*h
    
        
        end = time.clock() - start
        print ('%.2e sec.' % end)
        
        return value

Использование класса: вычисление значений

In [ ]:
listOfFunctions = [lambda t: np.exp(-t**2), lambda t: 2./ (1 + np.sin(np.exp(-t**2))),
                   lambda t: np.sinh(5./(2. + np.cos(t))), lambda t: 1./(1 + t**2)]

listOfAntiderivatives = [Integral() for i in range(len(listOfFunctions))]

for integral, function in zip(listOfAntiderivatives, listOfFunctions):
    integral.SetFunction(function)
    integral.SetNumBlocks(100)
    integral.SetLeftBorder(0.)

# после задания всех параметров пользуемся как функцией    
x0 = 5.
# выводим значения всех функций на экран
for i in range(len(listOfFunctions)):
    print ('#%d: Value at the point x0 = %.2f: %.2f ' % (i, x0, listOfAntiderivatives[i](x0)))

Использование класса: построение графиков

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

xData = np.linspace(0., 3., 50)

for element in listOfAntiderivatives:
    yData = [element(x) for x in xData]
    plt.figure()
    plt.plot(xData, yData, 'b-', linewidth=4)
    plt.grid('on')
    plt.show()

## Вопросы?